# 💻 Guia de Prática 6 💻

### 👉 Passos iniciais:

In [204]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: Erick Nathan M. Alves & Victor Sidnei Cotta
@data: 16/11/2022
"""



#Importando as bibliotecas:
import numpy as np # importando biblioteca numpy
import matplotlib.pyplot as plt # importando biblioteca para plotar as figuras
import control as ct  #importando biblioteca python control systems

plt.close('all') #comando para fechar todas janelas de plot

#Exibição dos gráficos em janela externa:
%matplotlib qt
#Exibição dos gráficos em no notebook:
#%matplotlib inline



#Parâmetros do sistema:
La = .154
L1 = .155
Lt = .270
d = .02
m = .005
rho = 1.23
c = 2.05
mi = 5
g = 9.81

#Constantes do modelo:
K1 = ((d*rho*c*La*L1)/(2*m*(((Lt**2)/12)+(d**2))))
K2 = ((g*d)/(((Lt**2)/12)+(d**2)))
K3 = ((mi*d**2)/(m*(((Lt**2)/12)+(d**2))))

#Dinâmica do sistema:
X0 = [0, 0] #condições iniciais

#Caracterizando o modelo, em espaço de estados:
def model_update(t, x, u, params):
    
    x1 = x[0] # posicao
    x2 = x[1] # velocidade

    #Retorna as derivadas:
    return [x2, ((K1*(np.cos(x1)**2)*u[0]) - ((K2*np.sin(x1)) + (K3*x2)))]

#Função que retorna o estado:
def model_output(t, x, u, params):
    return x

#Instanciando o sistema:
fanplate = ct.NonlinearIOSystem(model_update, model_output, states=2, name='fanplate', inputs=('u'), outputs=('x1','x2'))

### 👉 Item 2:

* Levando o sistema ao ponto de operação (27°):

In [205]:
#Parâmetros de simulação:
t = np.arange(0, 12, .01) #criando array de tempo
pop = 27 #Posição de equilíbrio em graus
u_eq = (((K2/K1)*np.sin(np.radians(pop)))/(np.cos(np.radians(pop))**2)) #sinal que leva ao ponto desejado
u = u_eq*np.ones(t.shape) #criando array com para o sinal

#Solucionando o sistema em si:
t, x = ct.input_output_response(fanplate, t, u, X0=X0)
y = x[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(1)
plt.rcParams['figure.figsize'] = (12, 5)
plt.subplot(2, 1, 1)
plt.plot(t, np.degrees(y),'red', label='$\\theta_{(t)}$')
plt.ylabel('$\\theta$ [°]')
plt.title('Levando o Sistema ao Ponto de Equilíbrio')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(t, u, 'green', label='$u_{(t)}$')
plt.ylabel('$u_{(t)}$')
plt.xlabel('Tempo [s]')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Aplicação de um degrau de +5° em torno do equilíbrio:

In [206]:
#Parâmetros de simulação:
X0 = [np.radians(27), 0]
Ad = 5 #Amplitude do degrau
ud = (((K2/K1)*np.sin(np.radians(pop+Ad)))/(np.cos(np.radians(pop+Ad))**2)) #sinal que aplica o degrau
td = np.arange(0, 25, .01) #criando array de tempo
#Criação do degrau:
u = np.hstack(((u_eq*np.ones(100)), (ud*np.ones(1200)), (u_eq*np.ones(1200))))    

#Solucionando o sistema em si:
td, xd = ct.input_output_response(fanplate, td, u, X0=X0)
yd = xd[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(2)
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.subplot(2, 1, 1)
plt.plot(td, np.degrees(yd),'red', label='$\\theta_{(t)}$')
plt.ylabel('$\\theta$ [°]')
plt.title('Resposta à Entrada em Degrau')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(td, u, 'green', label='$u_{d_{(t)}}$')
plt.ylabel('$u_{d_{(t)}}$')
plt.xlabel('Tempo [s]')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Obtenção dos parâmetros para modelo de Ziegler-Nichols, por resposta ao degrau positivo (borda de subida):

In [207]:
#Equilíbrio:
y0 = (pop*np.ones(td.shape))

#Reta tangente:
coef_ang = 3.3
coef_lin = 23.68
y1 = (coef_ang*td + coef_lin) #ajustada graficamente

#Atraso / tempo morto / fase:
A = 1 #obtido graficamente (há uma antecipação que não existe, o atraso é zero aqui, portanto o tempo morto também é zero)!!!!!!!!!!!!!!!

#Reta 0,63K:
y2 = ((pop+(0.63*Ad))*np.ones(td.shape))
B = (((pop+(0.63*Ad))-coef_lin)/coef_ang)

#Reta K:
y3 = ((pop+Ad)*np.ones(td.shape))
C = (((pop+Ad)-coef_lin)/coef_ang)

print("\n Resposta ao degrau (+5°): \tA = " + str(round(A, 4)) + "\t B = " + str(round(B, 4)) + "\t C = " + str(round(C, 4)))

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(3)
plt.rcParams['figure.figsize'] = (12, 5)
plt.plot(td, np.degrees(yd),'red',label='$\\theta_{(t)}$')
plt.plot(td, y0,'black',label='Equilíbrio')
plt.plot(td, y1,'pink',label='Tangente')
plt.plot(td, y2,'yellow',ls='--',label='0,63k')
plt.plot(td, y3,'blue',ls='--',label='k')
plt.scatter(A, pop, c='orange', linewidths=3, label='A')
plt.scatter(B, (pop+(0.63*Ad)), c='purple', linewidths=3, label='B')
plt.scatter(C, (pop+Ad), c='green', linewidths=3, label='C')
plt.ylabel('$\\theta$ [°]')
plt.xlabel('Tempo [s]')
plt.xlim(0, 10)
plt.legend(loc='lower right')
plt.ylim(26.9, 32.2)
plt.title('Resposta ao degrau com 3 parâmetros (+5°)')
plt.grid()
plt.show()


 Resposta ao degrau (+5°): 	A = 1	 B = 1.9606	 C = 2.5212


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Validação do modelo:

In [208]:
#Resposta ao degrau:
tau = (C-A)
print(f'\n tau: \t {round(tau, 4)}')
theta = 0
def sd(t, k, tau, theta):
    return (k*(1-np.exp((theta-t)/tau)))

#Criação de degraus:
udeg = []
kdeg = [-3, 3, -5, 8, 5]
for i in range(len(kdeg)):
    udeg.append((((K2/K1)*np.sin(np.radians(pop+kdeg[i])))/(np.cos(np.radians(pop+kdeg[i]))**2))*np.ones(1200))
uv = np.hstack(udeg)

tv = np.arange(0, 60, .01)

ysv = np.hstack(((pop+sd(tv[:1200], -3, tau, theta)), ((pop+kdeg[0])+sd(tv[:1200], 6, tau, theta)), ((pop+kdeg[1])+sd(tv[:1200], -8, tau, theta)), ((pop+kdeg[2])+sd(tv[:1200], 13, tau, theta)), ((pop+kdeg[3])+sd(tv[:1200], -3, tau, theta))))

#Parâmetros de simulação:
X0 = [np.radians(pop), 0]

#Solucionando o sistema em si:
tv, xv = ct.input_output_response(fanplate, tv, uv, X0=X0)
yv = np.degrees(xv[0])

#Função de transferência para um sistema de primeira ordem superamortecido:
s = ct.tf('s')
K = (np.radians(Ad)/(ud-u_eq)) #ganho estático do sistema
G = (K/((tau*s)+1))
print('\nG: ' + str(G))

tv, xft = ct.forced_response(G, T=tv, U=(uv-u_eq))
yft = (np.degrees(xft)+pop)

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(4)
plt.rcParams['figure.figsize'] = (12, 5)
plt.plot(tv, yv, 'red', label='Dinâmica do sistema')
plt.plot(tv, ysv,'green',label='Resposta Temporal Bibliográfica')
plt.plot(tv, yft,'blue',label='Resposta Forçada da FT Obtida')
plt.ylabel('$\\theta$ [°]')
plt.xlabel('Tempo [s]')
#plt.xlim(350, tf)
plt.legend()
#plt.ylim(70, 90)
plt.title('Validação do Modelo Obtido')
plt.grid()
plt.show()


 tau: 	 1.5212

G: 
  0.3245
-----------
1.521 s + 1



qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


### 👉 Item 3:

* Inclusão do atraso ao modelo obtido:

In [209]:
theta = 0.15

td1 = np.arange(0, 60, .01)

yd1 = np.hstack(((pop+sd(tv[:1200], -3, tau, theta)), ((pop+kdeg[0])+sd(tv[:1200], 6, tau, theta)), ((pop+kdeg[1])+sd(tv[:1200], -8, tau, theta)), ((pop+kdeg[2])+sd(tv[:1200], 13, tau, theta)), ((pop+kdeg[3])+sd(tv[:1200], -3, tau, theta))))

#Parâmetros de simulação:
X0 = [np.radians(pop), 0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(5)
plt.rcParams['figure.figsize'] = (12, 5)
plt.plot(tv, yv, 'red', label='Resposta em Malha Aberta')
plt.plot(tv, yd1,'blue',label='Reposta com atraso de 0.15s')
plt.ylabel('$\\theta$ [°]')
plt.xlabel('Tempo [s]')
plt.legend()
plt.title('Inserção de atraso ao modelo')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


### 👉 Itens 4 e 5:

* Incluindo atraso por aproximação de Padé (ordens: 1, 3, 5 e 9):

In [210]:
#Funções de transferência das ordens especificadas:
Gd = []
o_p = [1, 3, 5, 9]
for i in range(len(o_p)):
    #Aproximação de padé:
    N = ct.pade(0.15, o_p[i])
    #Função de transferência do atraso:
    Gd.append(ct.TransferFunction(np.array(N[0]), np.array(N[1])))
    print(f'\n Gd{o_p[i]}: \t {Gd[i]}')


 Gd1: 	 
-s + 13.33
----------
s + 13.33


 Gd3: 	 
-s^3 + 80 s^2 - 2667 s + 3.556e+04
----------------------------------
s^3 + 80 s^2 + 2667 s + 3.556e+04


 Gd5: 	 
-s^5 + 200 s^4 - 1.867e+04 s^3 + 9.956e+05 s^2 - 2.987e+07 s + 3.982e+08
------------------------------------------------------------------------
s^5 + 200 s^4 + 1.867e+04 s^3 + 9.956e+05 s^2 + 2.987e+07 s + 3.982e+08


 Gd9: 	 
-s^9 + 600 s^8 - 1.76e+05 s^7 + 3.285e+07 s^6 - 4.271e+09 s^5 + 3.986e+11 s^4 - 2.657e+13 s^3 + 1.215e+15 s^2 - 3.442e+16 s + 4.589e+17
---------------------------------------------------------------------------------------------------------------------------------------
s^9 + 600 s^8 + 1.76e+05 s^7 + 3.285e+07 s^6 + 4.271e+09 s^5 + 3.986e+11 s^4 + 2.657e+13 s^3 + 1.215e+15 s^2 + 3.442e+16 s + 4.589e+17



### 👉 Item 6:

* Simulação do sistema com atraso:

In [211]:
y_malha = []
u = u_eq*np.ones(t.shape) #criando array com para o sinal
for i in range(4):
    atraso = ct.tf2io(Gd[i], name='atraso', inputs='u', outputs='y')
    malha = ct.InterconnectedSystem(
        (atraso, fanplate), name='malha', 
        connections=(('atraso.u',), ('fanplate.u', 'atraso.y')), 
        inplist=('atraso.u'),
        outlist=('fanplate.x1', 'fanplate.x2')
    )
    X0 = np.zeros(o_p[i]+2)

    t, ym = ct.input_output_response(malha, t, u, X0)
    y_malha.append(ym[0])

#Parâmetros de simulação:
X0 = [0, 0]
tn = np.arange(0, 12, .01) #criando array de tempo
u = u_eq*np.ones(t.shape) #criando array com para o sinal

#Solucionando o sistema em si:
tn, x = ct.input_output_response(fanplate, tn, u, X0=X0)
y = x[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(6)
plt.rcParams['figure.figsize'] = (12, 5)
plt.plot(tn, np.degrees(y), 'pink', label='Sem atraso')
plt.plot(t, np.degrees(y_malha[0]), 'red', label='Padé 1ª ordem')
plt.plot(t, np.degrees(y_malha[1]),'green',label='Padé 3ª ordem')
plt.plot(t, np.degrees(y_malha[2]),'blue',label='Padé 5ª ordem')
plt.plot(t, np.degrees(y_malha[3]),'yellow',label='Padé 9ª ordem')
plt.ylabel('$\\theta$ [°]')
plt.xlabel('Tempo [s]')
plt.legend(loc='lower right')
plt.title('Comparação das aproximações por Padé')
plt.grid()
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


### 👉 Item 7:

* Comparação resposta ao degrau com atraso, por aproximação de Padé de 5ª ordem:

In [212]:
#Parâmetros de simulação:
X0 = [np.radians(pop), 0]

#Criação do degrau:
u = np.hstack(((ud*np.ones(1200)), (u_eq*np.ones(1200)))) 
t = np.arange(0, 24, .01) #criando array de tempo

#Solucionando o sistema em si:
t, xdeg = ct.input_output_response(fanplate, t, u, X0=X0)
ydeg = xdeg[0]

atraso = ct.tf2io(Gd[2], name='atraso', inputs='u', outputs='y')
malha = ct.InterconnectedSystem(
    (atraso, fanplate), name='malha', 
    connections=(('atraso.u',), ('fanplate.u', 'atraso.y')), 
    inplist=('atraso.u'),
    outlist=('fanplate.x1', 'fanplate.x2')
)
X0 = np.array([0, 0, 0, 0, 0, np.radians(pop), 0])

t, xm = ct.input_output_response(malha, t, u, X0)
y_md = xm[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(7)
plt.rcParams['figure.figsize'] = (12, 5)
plt.plot(t, np.degrees(ydeg), 'red', label='Resposta em Malha Aberta')
plt.plot(t, np.degrees(y_md),'blue',label='Resposta em atraso de 0.15s (Padé de 5ª ordem)')
plt.ylabel('$\\theta$ [°]')
plt.xlabel('Tempo [s]')
plt.legend(loc='lower right')
plt.title('Comparação com atraso à resposta ao degrau')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


### 👉 Item 8:

* Controlador Proporcional - Ziegler Nichols:

In [213]:
# Constantes via parâmetros de Ziegler Nichols - Tabelas 8.6 e 8.7 (Garcia, 2017):
kd = (np.radians(Ad)/(ud-u_eq))
Kc_pzn = (tau/(kd*theta))
print(f'\nKc: \t {round(Kc_pzn, 4)}')



Kc: 	 31.249


* Controlador Proporcional - CHR:

In [214]:
# Constantes via parâmetros de CHR - Tabela 8.9 (Garcia, 2017):
Kc_pchr = ((0.3*tau)/(kd*theta))
print(f'\nKc: \t {round(Kc_pchr, 4)}')


Kc: 	 9.3747


* Controlador Proporcional Integral - Ziegler Nichols:

In [215]:
# Constantes via parâmetros de Ziegler Nichols - Tabelas 8.6 e 8.7 (Garcia, 2017):
Kc_pizn = ((0.9*tau)/(kd*theta))
Ti_pizn = ((10*theta)/3)
print(f'\nKc: \t {round(Kc_pizn, 4)}')
print(f'\nTi: \t {round(Ti_pizn, 4)}')


Kc: 	 28.1241

Ti: 	 0.5


* Controlador Proporcional Integral - CHR:

In [216]:
# Constantes via parâmetros de CHR - Tabela 8.9 (Garcia, 2017):
Kc_pichr = ((0.6*tau)/(kd*theta))
Ti_pichr = (4*theta)
print(f'\nKc: \t {round(Kc_pichr, 4)}')
print(f'\nTi: \t {round(Ti_pichr, 4)}')


Kc: 	 18.7494

Ti: 	 0.6


### 👉 Item 10:

* Preparando respostas das malhas para r = [0 1 0 −1 0.5 1 −0.5 0]:

In [217]:
#Array de tempo:
t = np.arange(0, 64, .01)

#Preparando o sinal de referência:
r0 = [0, 1, 0, -1, 0.5, 1, -0.5, 0]
rs = []
us = []
for i in range(len(r0)):
    ri = np.radians(pop+(Ad*r0[i]))
    rs.append(ri*np.ones(800))
    us.append((((K2/K1)*np.sin(ri))/(np.cos(ri)**2))*np.ones(800))
r = np.hstack(rs)
u = np.hstack(us)

#Preparando o sinal de controle de operação:
u0 = u_eq*np.ones(t.shape)

#Preparando os controladores:

#Definição das funções de transferência:
ft_pzn = ct.tf(Kc_pzn, 1) # P - Ziegler Nichols
ft_pchr = ct.tf(Kc_pchr, 1) # P - CHR
ft_pizn1 = ct.tf(Kc_pizn, 1) # PI - Ziegler Nichols (Parte Proporcional)
ft_pizn2 = (Ti_pizn/s) # PI - Ziegler Nichols (Parte Integral)
ft_pichr1 = ct.tf(Kc_pichr, 1) # PI - CHR (Parte Proporcional)
ft_pichr2 = (Ti_pichr/s) # PI - CHR (Parte Integral)

#Definindo os subsistemas em formato de entradas e saídas:
sys_pzn = ct.tf2io(ft_pzn, name='pzn', inputs='u', outputs='y')
sys_pchr = ct.tf2io(ft_pchr, name='pchr', inputs='u', outputs='y')
sys_pizn1 = ct.tf2io(ft_pizn1, name='pizn1', inputs='u', outputs='y')
sys_pizn2 = ct.tf2io(ft_pizn2, name='pizn2', inputs='u', outputs='y')
sys_pichr1 = ct.tf2io(ft_pichr1, name='pichr1', inputs='u', outputs='y')
sys_pichr2 = ct.tf2io(ft_pichr2, name='pichr2', inputs='u', outputs='y')

# Atraso:
atraso = ct.tf2io(Gd[-2], name='atraso', inputs='u', outputs='y')

# Malha aberta:
X0 = [np.radians(pop), 0]
t, xout = ct.input_output_response(fanplate, t, u, X0)
ma = xout[0]


* Controlador Proporcional - Ziegler Nichols:

In [218]:
#Conexão da malha fechada:
closed_loop1 = ct.InterconnectedSystem(
    (fanplate, atraso, sys_pzn), name='closed_loop1', 
    connections=(('pzn.u', '-fanplate.x1'), ('atraso.u', 'pzn.y'), ('fanplate.u', 'atraso.y')), 
    inplist=('pzn.u', 'atraso.u'),
    inputs=('xref', 'u0'),
    outlist=('fanplate.x1', 'fanplate.x2', 'fanplate.u'),
    outputs=('x1', 'x2', 'u')
)

# Simulando Malha Fechada:
X0 = np.zeros(7)
t, xout = ct.input_output_response(closed_loop1, t, [r, u0], X0)
cpzn = xout[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(8)
plt.rcParams['figure.figsize'] = (12, 5)
plt.subplot(2, 1, 1)
plt.plot(t, np.degrees(ma), 'red', label='MA')
plt.plot(t, np.degrees(cpzn),'blue',label='PZN')
plt.ylabel('$\\theta$ [°]')
plt.legend(loc='lower right')
plt.ylim(18, 35)
plt.title('Malha Aberta X Controlador P - Ziegler Nichols')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(t, xout[2], 'green', label='$u_{(t)}$')
plt.ylabel('$u_{(t)}$')
plt.xlabel('Tempo [s]')
plt.grid()
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Controlador Proporcional - CHR:

In [219]:
#Conexão da malha fechada:
closed_loop2 = ct.InterconnectedSystem(
    (fanplate, atraso, sys_pchr), name='closed_loop2', 
    connections=(('pchr.u', '-fanplate.x1'), ('atraso.u', 'pchr.y'), ('fanplate.u', 'atraso.y')), 
    inplist=('pchr.u', 'atraso.u'),
    inputs=('xref', 'u0'),
    outlist=('fanplate.x1', 'fanplate.x2', 'fanplate.u'),
    outputs=('x1', 'x2', 'u')
)

# Simulando Malha Fechada:
X0 = np.zeros(7)
t, xout = ct.input_output_response(closed_loop2, t, [r, u0], X0)
cpchr = xout[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(9)
plt.rcParams['figure.figsize'] = (12, 5)
plt.subplot(2, 1, 1)
plt.plot(t, np.degrees(ma), 'red', label='MA')
plt.plot(t, np.degrees(cpchr),'blue',label='PCHR')
plt.ylabel('$\\theta$ [°]')
plt.legend(loc='lower right')
plt.ylim(18, 35)
plt.title('Malha Aberta X Controlador P - CHR')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(t, xout[2], 'green', label='$u_{(t)}$')
plt.ylabel('$u_{(t)}$')
plt.xlabel('Tempo [s]')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Controlador Proporcional Integral - Ziegler Nichols:

In [220]:
#Conexão da malha fechada:
closed_loop3 = ct.InterconnectedSystem(
    (fanplate, atraso, sys_pizn1, sys_pizn2), name='closed_loop3', 
    connections=(('pizn1.u', '-fanplate.x1'), ('atraso.u', 'pizn1.y'), ('pizn2.u', '-fanplate.x1'), ('atraso.u', 'pizn2.y'), ('fanplate.u', 'atraso.y')), 
    inplist=('pizn1.u', 'pizn2.u', 'atraso.u'),
    inputs=('xref1', 'xref2', 'u0'),
    outlist=('fanplate.x1', 'fanplate.x2', 'fanplate.u'),
    outputs=('x1', 'x2', 'u')
)

# Simulando Malha Fechada:
X0 = np.zeros(7)
t, xout = ct.input_output_response(closed_loop3, t, [r, r, u0], X0)
cpizn = xout[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(10)
plt.rcParams['figure.figsize'] = (12, 5)
plt.subplot(2, 1, 1)
plt.plot(t, np.degrees(ma), 'red', label='MA')
plt.plot(t, np.degrees(cpizn),'blue',label='PCHR')
plt.ylabel('$\\theta$ [°]')
plt.legend(loc='lower right')
plt.ylim(18, 35)
plt.title('Malha Aberta X Controlador PI - Ziegler Nichols')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(t, xout[2], 'green', label='$u_{(t)}$')
plt.ylabel('$u_{(t)}$')
plt.xlabel('Tempo [s]')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Controlador Proporcional Integral - CHR:

In [221]:
#Conexão da malha fechada:
closed_loop4 = ct.InterconnectedSystem(
    (fanplate, atraso, sys_pichr1, sys_pichr2), name='closed_loop4', 
    connections=(('pichr1.u', '-fanplate.x1'), ('atraso.u', 'pichr1.y'), ('pichr2.u', '-fanplate.x1'), ('atraso.u', 'pichr2.y'), ('fanplate.u', 'atraso.y')), 
    inplist=('pichr1.u', 'pichr2.u', 'atraso.u'),
    inputs=('xref1', 'xref2', 'u0'),
    outlist=('fanplate.x1', 'fanplate.x2', 'fanplate.u'),
    outputs=('x1', 'x2', 'u')
)

# Simulando Malha Fechada:
X0 = np.zeros(7)
t, xout = ct.input_output_response(closed_loop4, t, [r, r, u0], X0)
cpichr = xout[0]

#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(11)
plt.rcParams['figure.figsize'] = (12, 5)
plt.subplot(2, 1, 1)
plt.plot(t, np.degrees(ma), 'red', label='MA')
plt.plot(t, np.degrees(cpichr),'blue',label='PCHR')
plt.ylabel('$\\theta$ [°]')
plt.legend(loc='lower right')
plt.ylim(18, 35)
plt.title('Malha Aberta X Controlador PI - CHR')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(t, xout[2], 'green', label='$u_{(t)}$')
plt.ylabel('$u_{(t)}$')
plt.xlabel('Tempo [s]')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Comparação entre os controladores:

In [222]:
#Plotando o resultado da simulação-------------------------------------------------------------------
plt.figure(12)
plt.rcParams['figure.figsize'] = (12, 5)
plt.plot(t, np.degrees(ma), 'pink', label='MA')
plt.plot(t, np.degrees(cpzn),'red',label='PZN')
plt.plot(t, np.degrees(cpchr),'green',label='PCHR')
plt.plot(t, np.degrees(cpizn),'blue',label='PIZN')
plt.plot(t, np.degrees(cpichr),'yellow',label='PICHR')
plt.ylabel('$\\theta$ [°]')
plt.legend(loc='lower right')
plt.ylim(18, 35)
plt.title('P X PI (Ziegler Nichols X CHR)')
plt.grid()
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


* Rejeição a perturbação:

### 👉 Item 12:

* Performance da malha fechada: